# Merge the walking network and extracted crossings from osm, project sidewalk, traffic signs and curb heights

In [ ]:
# Select where to run notebook: "azure" or "local"
my_run = "azure"

In [ ]:
import geopandas as gpd
import pandas as pd

from shapely.ops import split
from tqdm import tqdm

import settings as st

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf

In [ ]:
os.system('sudo blobfuse /home/azureuser/cloudfiles/code/blobfuse/sidewalk --tmp-path=/mnt/resource/blobfusetmp --config-file=/home/azureuser/cloudfiles/code/blobfuse/fuse_connection_sidewalk.cfg -o attr_timeout=3600 -o entry_timeout=3600 -o negative_timeout=3600 -o allow_other -o nonempty')

In [ ]:
# Import network and crossings
CRS = st.CRS

gdf_network = gpd.read_file(cf.output_file_widths).to_crs(crs=CRS)
gdf_osm_crossings = gpd.read_file(cf.output_osm_crossings).to_crs(crs=CRS)
gdf_project_sidewalk_crossings = gpd.read_file(cf.output_project_sidewalk_crossings).to_crs(crs=CRS)
gdf_traffic_sign_crossings = gpd.read_file(cf.output_traffic_sign_crossings).to_crs(crs=CRS)
gdf_bike_network = gpd.read_file(cf.output_bike_network_cut).to_crs(crs=CRS)
gdf_walk_bike_connections = gpd.read_file(cf.output_walk_bike_connections_base + '_max_height_0.04.gpkg').to_crs(crs=CRS)

gdf_curb_crossings_4cm = gpd.read_file(cf.output_curb_crossings_base + '_max_height_0.04.gpkg').to_crs(crs=CRS)
gdf_curb_crossings_6cm = gpd.read_file(cf.output_curb_crossings_base + '_max_height_0.06.gpkg').to_crs(crs=CRS)
gdf_curb_crossings_8cm = gpd.read_file(cf.output_curb_crossings_base + '_max_height_0.08.gpkg').to_crs(crs=CRS)

In [ ]:
# Set crossing and connection columns columns
gdf_osm_crossings['crossing_type'] = 'osm'
gdf_project_sidewalk_crossings['crossing_type'] = 'project_sidewalk'
gdf_traffic_sign_crossings['crossing_type'] = 'traffic_sign'

gdf_curb_crossings_4cm['curb_height_max'] = 0.04
gdf_curb_crossings_6cm['curb_height_max'] = 0.06
gdf_curb_crossings_8cm['curb_height_max'] = 0.08

gdf_curb_crossings = pd.concat([gdf_curb_crossings_4cm, gdf_curb_crossings_6cm, gdf_curb_crossings_8cm])
gdf_curb_crossings['crossing_type'] = 'curb_height'

gdf_crossings = pd.concat([gdf_osm_crossings, gdf_project_sidewalk_crossings, 
                     gdf_traffic_sign_crossings, gdf_curb_crossings])

In [ ]:
# Get intersections between bike network and crossings and add as nodes to the network
df, geoms = pd.DataFrame(columns=gdf_crossings.columns), []
for i, row in tqdm(gdf_crossings.iterrows(), total=len(gdf_crossings)):
    pos_intersections = row['geometry'].intersection(gdf_bike_network['geometry'])
    intersections = pos_intersections.unique()[1:]
    if intersections:
        for intersection in intersections:
            splits = split(row['geometry'], intersection).geoms
            for sub_edge in splits:
                df.loc[len(df)] = row
                geoms.append(sub_edge)
    else:
        df.loc[len(df)] = row
        geoms.append(row['geometry'])

gdf_crossings_new = gpd.GeoDataFrame(df, geometry=geoms, crs=CRS)
gdf_crossings_new['length'] = gdf_crossings_new.geometry.length

In [ ]:
# Set edge type columns
gdf_network['crossing'] = 'No'
gdf_bike_network['crossing'] = 'No'
gdf_crossings_new['crossing'] = 'Yes'
gdf_walk_bike_connections['crossing'] = 'No'

gdf_network['walk_bike_connection'] = 'No'
gdf_bike_network['walk_bike_connection'] = 'No'
gdf_crossings_new['walk_bike_connection'] = 'Yes'
gdf_walk_bike_connections['walk_bike_connection'] = 'Yes'

In [ ]:
# Merge walking network and crossings
gdf_walk_bike_network_including_crossings = pd.concat([gdf_network, gdf_bike_network, gdf_crossings_new, gdf_walk_bike_connections])

# Export network including crossings
gdf_walk_bike_network_including_crossings.to_file(cf.output_basic_network_including_crossings, driver='GPKG')